# Fetch ALL OSM Data - Comprehensive Tag Fetching

**Purpose**: Fetch ALL OpenStreetMap features within a radius to inspect raw data structure and decide on classification strategy.

This notebook:
- Fetches everything using `features_from_point()` with comprehensive tags (amenity, leisure, shop, tourism, highway, railway, building, landuse, office, sport)
- Shows data structure, columns, and sample rows
- Allows you to explore tags and decide how to classify POIs
- Exports to shapefile/GeoJSON for inspection

**Note**: OSMnx requires tags parameter, so we use `True` for all major tag categories to fetch everything.


In [1]:
import os
import sys
import ssl
import warnings
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == 'notebook' else Path.cwd()
sys.path.insert(0, str(project_root))

# SSL Configuration (for corporate proxies)
ssl._create_default_https_context = ssl._create_unverified_context
os.environ['PYTHONHTTPSVERIFY'] = '0'
os.environ['CURL_CA_BUNDLE'] = ''
os.environ['REQUESTS_CA_BUNDLE'] = ''

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Import geospatial libraries
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import json

# Configure OSMnx
warnings.filterwarnings('ignore')
ox.settings.log_console = True
ox.settings.use_cache = True
ox.settings.timeout = 300

print("✅ Libraries loaded")
print(f"📁 Project root: {project_root}")


✅ Libraries loaded
📁 Project root: /Users/nitish.ranjan/Documents/AiDash/Educational/research/locality-lens


## Step 1: Fetch ALL OSM Data (No Tag Filtering)


In [2]:
# Configuration
LOCATION = "Indiranagar, Bangalore"  # Change this to test different locations
RADIUS_M = 2000  # 2km radius

# Option 1: Geocode first (if using address)
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="locality_lens", timeout=10)

try:
    location = geolocator.geocode(LOCATION)
    if location:
        lat, lon = location.latitude, location.longitude
        print(f"📍 Geocoded: {LOCATION}")
        print(f"   Coordinates: {lat:.6f}, {lon:.6f}")
    else:
        raise ValueError("Geocoding failed")
except Exception as e:
    print(f"⚠️ Geocoding failed: {e}")
    print("Using default coordinates for Indiranagar, Bangalore")
    lat, lon = 12.9732913, 77.6404672

print(f"\n🔍 Fetching ALL OSM features within {RADIUS_M}m radius...")
print("   (This may take 10-30 seconds depending on data density)")

# Fetch EVERYTHING using comprehensive tags
# OSMnx requires tags, so we use very broad tags to get everything
all_features = ox.features_from_point(
    center_point=(lat, lon),
    dist=RADIUS_M,
    tags = {
        # Fetch all amenities (catch regional variations)
        'amenity': True,  # Then filter in-memory for what you need
        
        # Fetch all leisure (catch variations)
        'leisure': True,
        
        # Fetch all shops
        'shop': True,
        
        # Fetch specific transportation (these are standardized)
        'highway': ['bus_stop'],
        'railway': ['station', 'subway', 'subway_entrance', 'platform', 'light_rail', 'tram'],
        
        # Fetch specific tourism
        'tourism': ['hotel', 'hostel', 'guest_house', 'artwork', 'attraction'],
    }
)

print(f"\n✅ Fetched {len(all_features)} total features")
print(f"📊 Geometry types: {all_features.geom_type.value_counts().to_dict()}")


⚠️ Geocoding failed: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Indiranagar%2C+Bangalore&format=json&limit=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))
Using default coordinates for Indiranagar, Bangalore

🔍 Fetching ALL OSM features within 2000m radius...
   (This may take 10-30 seconds depending on data density)

✅ Fetched 3369 total features
📊 Geometry types: {'Point': 2771, 'Polygon': 533, 'LineString': 64, 'MultiPolygon': 1}


In [3]:
# Let's inspect the different geometry types present in the all_features GeoDataFrame

geometry_types = all_features.geom_type.value_counts()
print("Geometry Types Found in OSM Data:")
for geom_type, count in geometry_types.items():
    print(f" - {geom_type}: {count}")

Geometry Types Found in OSM Data:
 - Point: 2771
 - Polygon: 533
 - LineString: 64
 - MultiPolygon: 1


In [4]:
import numpy as np

def determine_poi_type(row):
    for col in ["amenity", "leisure", "shop", "highway", "railway", "tourism"]:
        value = row.get(col)
        if pd.notnull(value):
            return value
    return np.nan

all_features['poi_type'] = all_features.apply(determine_poi_type, axis=1)

In [5]:
import geopandas as gpd
from scipy.spatial import cKDTree
import numpy as np

# Drop features with missing name first
all_features = all_features.dropna(subset=['name']).reset_index(drop=True)

# Create poi_type if it doesn't exist (based on your classification logic)
# Assuming you have a function to create poi_type, or it's already there

# OPTIMIZATION 1: Group by name and poi_type first (reduces search space dramatically)
all_features['name_poi_key'] = all_features['name'].astype(str) + '_' + all_features['poi_type'].astype(str)

# Get centroid coordinates for distance calculation
all_features['centroid_x'] = all_features.geometry.centroid.x
all_features['centroid_y'] = all_features.geometry.centroid.y

# OPTIMIZATION 2: Use spatial indexing with grouping
to_drop = set()

# Group by name_poi_key - only check duplicates within same group
for name_poi, group in all_features.groupby('name_poi_key'):
    if len(group) == 1:
        continue  # No duplicates possible
    
    # Extract coordinates for this group
    coords = np.column_stack([group['centroid_x'].values, group['centroid_y'].values])
    
    # OPTIMIZATION 3: Use KDTree for fast spatial queries
    tree = cKDTree(coords)
    
    # Find all pairs within 200m (0.0018 degrees ≈ 200m at equator)
    # More accurate: use haversine, but for speed, approximate with degrees
    # 200m ≈ 0.0018 degrees at equator, adjust for your latitude
    max_distance_deg = 200 / 111000  # Rough conversion (1 degree ≈ 111km)
    
    # Query all points within max_distance_deg
    pairs = tree.query_pairs(max_distance_deg, output_type='ndarray')
    
    if len(pairs) > 0:
        # Keep first index of each pair, drop second
        indices_to_drop = group.iloc[pairs[:, 1]].index.tolist()
        to_drop.update(indices_to_drop)

# Drop duplicates and clean up helper columns
all_features = all_features.drop(index=list(to_drop)).drop(
    columns=['name_poi_key', 'centroid_x', 'centroid_y']
).reset_index(drop=True)

In [6]:
all_features.shape

(2560, 338)

In [7]:
all_features['poi_type'].value_counts()

poi_type
restaurant          300
clothes             145
fast_food           125
place_of_worship    111
cafe                104
                   ... 
music_school          1
lighting              1
taxi                  1
spa;saloon;cafe       1
common                1
Name: count, Length: 190, dtype: int64